In [0]:
#imports

from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.datasets as dset

In [0]:
#Allow the optimization 
cudnn.benchmark = True

In [0]:
imageSize = 64  
batchSize = 64
nz = 100                # size of the latent z vector
ngf = 64                
ndf = 64                
ngpu = 1

In [0]:
#load MNIST dataset

dataset = dset.MNIST(root='data/mnist', download=True,
                           transform=transforms.Compose([
                               transforms.Resize(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, ), (0.5, )),
                           ]))
nc = 1   # number of channels in MNIST dataset is 1

In [0]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
                                         shuffle=True, num_workers=2) 

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.02)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            m.weight.data.normal_(0, 0.02)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.normal_(1, 0.02)
            if m.bias is not None:
                m.bias.data.zero_()


In [0]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        initialize_weights(self)

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.gpu))
        else:
            output = self.main(input)

        return output

In [0]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

In [0]:
netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)

In [80]:
print(netG)
print(netD)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [0]:
criterion = nn.BCELoss()

In [0]:
fixed_noise = torch.randn(batchSize, nz, 1, 1, device=device)
fake_label = 0
real_label = 1

optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG= optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [92]:
for epoch in range(20):
    for i, data in enumerate(dataloader, 0):

        netD.zero_grad()
        trainData = data[0].to(device)
        batch_size = trainData.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(trainData)
        D_err_real = criterion(output, label)
        D_err_real.backward()
        D_x = output.mean().item()


        #train with fake data
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        D_err_fake = criterion(output, label)
        D_err_fake.backward()
        D_G_z1 = output.mean().item()
        D_err = D_err_real + D_err_fake
        optimizerD.step()

        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake)
        G_err = criterion(output, label)
        G_err.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('Epoch: ', epoch, ' Iteration: ', i, 'Generator loss: ', G_err.item(),
              ' Discriminator loss ', D_err.item()  )
        if i % 100 == 0:
            vutils.save_image(trainData, 'trainImage.png', normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(), 'fakeImage.png', normalize=True)

    torch.save(netG.state_dict(), 'netG_torch_%d.pth' % epoch)
    torch.save(netD.state_dict(), 'netD_torch_%d.pth' % epoch)    

Epoch:  19  Iteration:  901 Generator loss:  61.445045471191406  Discriminator loss  1.4901164746561335e-08
Epoch:  19  Iteration:  902 Generator loss:  61.62031555175781  Discriminator loss  0.0
Epoch:  19  Iteration:  903 Generator loss:  61.19973373413086  Discriminator loss  1.4901162970204496e-08
Epoch:  19  Iteration:  904 Generator loss:  61.49208068847656  Discriminator loss  1.862645371275562e-09
Epoch:  19  Iteration:  905 Generator loss:  61.446372985839844  Discriminator loss  1.862645371275562e-09
Epoch:  19  Iteration:  906 Generator loss:  62.05714416503906  Discriminator loss  1.862645371275562e-09
Epoch:  19  Iteration:  907 Generator loss:  61.31952667236328  Discriminator loss  7.450581485102248e-09
Epoch:  19  Iteration:  908 Generator loss:  61.46104431152344  Discriminator loss  1.3038517820973539e-08
Epoch:  19  Iteration:  909 Generator loss:  61.27679443359375  Discriminator loss  5.587936335871291e-09
Epoch:  19  Iteration:  910 Generator loss:  61.14089965820